In [9]:
from langchain_groq import ChatGroq
import os
os.environ["GROQ_API_KEY"]="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"
model=ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",temperature=0.2)


In [ ]:
from langchain.agents import initialize_agent, AgentType,load_tools
import os
os.environ["SERPAPI_API_KEY"]="your api here"
tools=load_tools(["serpapi"],llm=model)
agent=initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Who is Rishiraj Bal of kiit?get a detailed report of him about his everything give links to his accounts and also track his github account named https://github.com/Rishirajbal")

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

wiki = WikipediaAPIWrapper(top_k_results=3, doc_content_char_max=200)
result = wiki.run("Who is rahul bal jnli?")
tool=WikipediaQueryRun(api_wrapper=wiki, llm=model)
tool.run("Who is christopher nolan?")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer
loader=WebBaseLoader(web_path="https://python.langchain.com/docs/integrations/tools/",
bs_kwargs=dict(
    parse_only=SoupStrainer(class_="theme-doc-markdown markdown")))

result=loader.load()
result    

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer

loader = WebBaseLoader(
    web_path="https://python.langchain.com/docs/integrations/tools/",
    bs_kwargs=dict(parse_only=SoupStrainer(class_="theme-doc-markdown markdown"))
)

result = loader.load()

os.environ["GROQ_API_KEY"] = "your api key"

model = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.2)

split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = split.split_documents(result)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(text, embeddings)

prompt = ChatPromptTemplate.from_template(
    """You are an intelligent, talkative, and precise AI.
You will answer the user's question based on the following context:
<context>
{context}
</context>

Question: {input}

Respond in a friendly and informative way. Be accurate, as you're in a competition with another AI."""
)

chain = create_stuff_documents_chain(model, prompt)
retriever = db.as_retriever()
chain2 = create_retrieval_chain(retriever, chain)

response = chain2.invoke({"input": "What is langchain?"})
print(response["answer"])
